In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import holoviews as hv
from holoviews import opts, dim
import pickle
import molecule_counting as molco
from scipy import stats
import importlib

In [ ]:
hv.extension("bokeh")

# Load data

In [ ]:
# with open('190311photobleaching.pickle', 'rb') as f:
# with open('all_data.pickle', 'rb') as f:
# with open('190313photobleaching_noflatcorr.pickle', 'rb') as f:
# with open('190326photobleaching_flatcorr_fluoronly.pickle', 'rb') as f:
with open(
    "/n/groups/paulsson/nao5/molecule-counting/190604_mothermachine.pickle", "rb"
) as f:
    d = pickle.load(f)
list(d.keys())

In [ ]:
importlib.reload(molco)

mvenus_dir = "/n/scratch2/jqs1/190604/mVenus_YFP_100ms_10pct.nd2"
mkate50_dir = "/n/scratch2/jqs1/190604/mkate_mcherry_100ms_50pct.nd2"
mkate100_dir = "/n/scratch2/jqs1/190604/mkate_mcherry_100ms_100pct.nd2"
gfp_dir = "/n/scratch2/jqs1/190604/GFP_FITC_100ms_60pct.nd2"

mkate50_bad_fov = [0, 3, 5, 6, 7]
mkate100_bad_fov = [1, 8, 10, 17, 19, 20]
mvenus_bad_fov = [0]
# mvenus_bad_fov = [0,8,9,10,11,12,13,14,15,16,17]
gfp_bad_fov = [8, 10, 11, 12, 13, 14, 15, 16, 17]

# data = molco.process_df(d,mkate50_dir,mkate50_bad_fov,6,1.8,.9)
data = molco.process_df(d, mkate100_dir, mkate100_bad_fov, 6, 1.3, 0.90)
# data = molco.process_df(d,mvenus_dir,mvenus_bad_fov,6,1.5,.9)
# data = molco.process_df(d,gfp_dir,gfp_bad_fov,6,1.2,.95)

In [ ]:
%%output size=250
traces = data.iloc[:, :950].values
idxs = np.random.permutation(traces.shape[0])
downsample = 100  # set to 1 to show all traces (instead odata.k.hist()f 10%); this will make your browser slow
curves = [
    {"x": np.arange(traces.shape[1]), "y": traces[i], "i": idxs[i]}
    for i in range(traces.shape[0] // downsample)
]
hv.Contours(curves, vdims=["i"]).options(color_index="i", cmap="Category20", logy=True)

In [ ]:
import segmentation

%output size=200
fov = 2
img = np.asarray(d[mkate100_dir][fov]["segmentation_frame"])
labels = np.asarray(d[mkate100_dir][fov]["labels"])
# measurements, regionprops, labels, img = data[ '/n/scratch2/jqs1/190604/GFP_FITC_100ms_60pct.nd2'][4]
labels = segmentation.permute_labels(labels)
hv.Image(img / img.max()).options(cmap="hot") + hv.Image(labels)

In [ ]:
importlib.reload(molco)
# prop_dict = {'area': [data.area.median() - 2*data.area.std(), data.area.median() + 2*data.area.std()],
#               0: [2000, 4500], t_end-1: [500,1000],
#              'k': [.003,.0045]}
# prop_dict = {
#             'centroid_y': [0, 2000],
#              'centroid_x': [0, 2000],
#              'area': [data.area.median() - 2*data.area.std(), data.area.median() + 2*data.area.std()],
#              0: [500, 6000],'k': [.002,.01]}
prop_dict = {
    "centroid_y": [400, 1600],
    "centroid_x": [700, 1800],
    "area": [
        data.area.median() - 2 * data.area.std(),
        data.area.median() + 2 * data.area.std(),
    ],
    0: [500, 6000],
    "k": [0.012, 0.02],
}  # , 1150:[0,350]}
df = molco.filter_df(data, prop_dict)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
ax1.hist(df.k, bins=30)
ax2.hist(df[0], bins=30)

q = 1

f_predicted, fbar, p, mu, sigma2, y = molco.fluct_plot(df, n=3)

# plt.figure(figsize=(12,8))
# yn = y.apply(lambda col: col/col.iloc[0],axis=1)
# plt.semilogy(yn.T.values[:,::20])
# plt.ylim(bottom=1e-3,top=1.1)
# plt.xlim(left=0)

plt.figure(figsize=(10, 8))
yn = y.apply(lambda col: col / col.iloc[0], axis=1)
tp = np.arange(yn.shape[1]) / 10
plt.semilogy(tp, yn.T.values[:, ::10], linewidth=3)
plt.semilogy(tp, mu / mu[0], "b", linewidth=7, alpha=0.7, label=r"$\mu$")
plt.xlabel("Time", fontsize=28)
plt.ylabel("Fluorescence", fontsize=28)
plt.legend(fontsize=20)
plt.ylim(bottom=1e-3, top=1.1)
plt.xlim(left=0)

plt.figure(figsize=(12, 8))
molco.kmaps(df)

In [ ]:
plt.figure(figsize=(12, 10))
plt.rc("xtick", labelsize=24)
plt.rc("ytick", labelsize=24)
t = 0.1 * np.arange(y.shape[1])
plt.semilogy(t, y.T.values[:, ::20])
plt.ylim(bottom=1e1, top=6e3)
plt.xlim(left=0)
plt.xlabel("Time (seconds)", fontsize=28)
plt.ylabel("Fluorescence (A.U.)", fontsize=28)
# plt.title('Photobleach Time Series')